
# NumPy 点云 / 三维重建实战笔记

## 1️⃣ 批量点到点距离

给定两组点云 `P` 和 `Q`，计算每对对应点的欧氏距离：

```python
import numpy as np

# 随机点云
N = 1000
P = np.random.rand(N,3)
Q = np.random.rand(N,3)

# 批量点到点距离
distances = np.linalg.norm(P - Q, axis=1)  # shape = (N,)
```

* `axis=1` → 每行计算向量长度
    
* 输出 `distances[i]` 是 `P[i]` 到 `Q[i]` 的距离
    

* * *

## 2️⃣ 点到平面距离

平面方程：`ax + by + cz + d = 0`，点 `p = (x,y,z)` 到平面距离：

$$\text{dist} = \frac{|a x + b y + c z + d|}{\sqrt{a^2+b^2+c^2}}$$

```python
# 平面参数
plane_normal = np.array([0,0,1])  # 法向量
d = -0.5

# 批量点到平面
numerator = np.abs(P @ plane_normal + d)
denominator = np.linalg.norm(plane_normal)
distances_to_plane = numerator / denominator
```

* `P @ plane_normal` → 点到平面法向量方向投影
    
* 广播 + np.abs() → 批量处理
    

* * *

## 3️⃣ 批量法向量与夹角

三角形顶点 `v0, v1, v2`，法向量：

$$\mathbf{n} = \frac{(v1-v0) \times (v2-v0)}{\|(v1-v0)\times(v2-v0)\|}$$

与某个方向（如 z 轴）夹角：

```python
v0 = np.random.rand(N,3)
v1 = np.random.rand(N,3)
v2 = np.random.rand(N,3)

edge1 = v1 - v0
edge2 = v2 - v0
normals = np.cross(edge1, edge2)
unit_normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

z_axis = np.array([0,0,1])
cos_theta = unit_normals @ z_axis  # shape = (N,)
angles_deg = np.degrees(np.arccos(np.clip(cos_theta, -1, 1)))
```

* `np.clip` 防止浮点误差导致 arccos 出错
    
* `angles_deg[i]` → 第 i 个三角形法向量与 z 轴夹角（度数）
    

* * *

## 4️⃣ 批量旋转矩阵 + 平移

给点云 `P`，刚体变换 `R`、`t`：

```python
# 随机旋转矩阵 R (3x3) & 平移向量 t
R = np.eye(3)  # 示例单位矩阵
t = np.array([1,2,3])

# 批量旋转 + 平移
P_transformed = P @ R.T + t
```

* `@ R.T` → 因为 P 是行向量
    
* 广播自动加上 t
    
* 可直接批量处理任意点云
    

* * *

## 5️⃣ ICP 最小二乘刚体求解（点对齐）

给源点云 `P` 和目标点云 `Q`，求刚体变换 `(R,t)` 最小化：

$$\min_{R,t} \sum_i \| R P_i + t - Q_i \|^2$$

### NumPy 实现示例（Kabsch 算法）

```python
# 中心化
P_centered = P - P.mean(axis=0)
Q_centered = Q - Q.mean(axis=0)

# 协方差矩阵
H = P_centered.T @ Q_centered  # shape (3,3)

# SVD 分解
U, S, Vt = np.linalg.svd(H)
R = Vt.T @ U.T

# 保证旋转矩阵不翻转
if np.linalg.det(R) < 0:
    Vt[2,:] *= -1
    R = Vt.T @ U.T

# 平移向量
t = Q.mean(axis=0) - P.mean(axis=0) @ R.T
```

* 输出 `R` / `t` → 最小二乘刚体变换
    
* 可以直接用于批量点云对齐
    

* * *

## 6️⃣ 总结

| 功能 | 公式 / 方法 | NumPy 技巧 |
| --- | --- | --- |
| 点到点距离 | $\|P_i - Q_i\|$ | `np.linalg.norm(P-Q, axis=1)` |
| 点到平面距离 | ( | n \cdot P + d |
| 三角形法向量 | ( (v1-v0) \times (v2-v0) / |  |
| 法向量夹角 | $\arccos(n \cdot z)$ | `unit_normals @ z_axis` |
| 批量旋转+平移 | $P R^T + t$ | 行向量矩阵乘法 + 广播 |
| ICP 刚体求解 | Kabsch 算法 | `SVD` + 中心化 + 平移 |

* * *

小白菜，这份笔记把你 **NumPy 批量几何计算 + 点云变换 + ICP 刚体求解** 都整理好了，动手就能用。

如果你愿意，我可以再给你画一个 **完整流程图**，从 **点云 → 法向量 → 距离 → 批量变换 → ICP 对齐**，把整个 VGGT/三维重建核心流程可视化，看一眼就懂。

你想看吗？